In [1]:
import torch
import torch.nn as nn
inputs = torch.Tensor(1,1,28,28) # batch Size, channel, height, width

In [2]:
inputs.shape

torch.Size([1, 1, 28, 28])

In [3]:
conv1 = nn.Conv2d(1, 32, 3, padding=1)
pool = nn.MaxPool2d(2)
conv2 = nn.Conv2d(32, 64, 3, padding=1)


In [4]:
out = conv1(inputs)
out.shape

torch.Size([1, 32, 28, 28])

In [5]:
out = pool(out)
out.shape

torch.Size([1, 32, 14, 14])

In [6]:
out=conv2(out)
out.shape

torch.Size([1, 64, 14, 14])

In [7]:
out = pool(out)
out.shape

torch.Size([1, 64, 7, 7])

In [8]:
out.size(1)

64

In [9]:
out = out.view(out.size(0), -1) # 배치사이즈는 그대로 하고 나머지는 펼쳐서 넣어라.
out.shape

torch.Size([1, 3136])

In [10]:
fc = nn.Linear(3136, 10)
out = fc(out)

In [11]:
out

tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],
       grad_fn=<AddmmBackward>)

In [12]:
out.shape 


torch.Size([1, 10])

# 구현

In [13]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(42)
if device =='cuda':
    torch.cuda.manual_seed_all(42)

In [15]:
# parameters
learning_rate = 0.001
epochs = 15
batch_size = 100

In [16]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                         train = True,
                         transform=transforms.ToTensor(),
                         download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                        train=False,
                        transform = transforms.ToTensor(),
                        download=True)

In [17]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size = batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [18]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
    
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
    
        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out=self.fc2(out)
        return out

In [19]:
model = CNN().to(device)
value = torch.Tensor(1,1,28,28).to(device)
print(model(value).shape)

torch.Size([1, 10])


In [20]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [21]:
# Training
total_batch = len(data_loader)

for epoch in range(epochs):
    avg_cost = 0
    
    for X, y in data_loader:
        X = X.to(device)
        y = y.to(device)
        
        optimizer.zero_grad() # 반드시 넣어야 한다
        hypothesis = model(X)
        cost = criterion(hypothesis, y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('Epoch : {:4d}/{} Cost : {}'.format(epoch+1,epochs, avg_cost))
print('Learning Finished')

Epoch :    1/15 Cost : 0.16829709708690643
Epoch :    2/15 Cost : 0.04154154658317566
Epoch :    3/15 Cost : 0.029126547276973724
Epoch :    4/15 Cost : 0.021229883655905724
Epoch :    5/15 Cost : 0.017771583050489426
Epoch :    6/15 Cost : 0.014171542599797249
Epoch :    7/15 Cost : 0.012186538428068161
Epoch :    8/15 Cost : 0.009611829183995724
Epoch :    9/15 Cost : 0.007756589446216822
Epoch :   10/15 Cost : 0.009729191660881042
Epoch :   11/15 Cost : 0.006594821810722351
Epoch :   12/15 Cost : 0.0056308298371732235
Epoch :   13/15 Cost : 0.0067752655595541
Epoch :   14/15 Cost : 0.006003663409501314
Epoch :   15/15 Cost : 0.005530753172934055
Learning Finished


In [22]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy : ', accuracy.item())

C:\Users\JeongMyeong\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\JeongMyeong\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy :  0.9795999526977539
